# Final Project: Decide on global dual sourcing strategy

team: \
David Yang \
Jack Chen \
Joyce Wu

WARNING: execution time is too long.

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('final project 2024.csv')

In [3]:
data.head(10)

,period,demand
0,1,21
1,2,81
2,3,32
3,4,58
4,5,47
5,6,49
6,7,66
7,8,29
8,9,55
9,10,39


In [4]:
s = 10 # sales price
c_m = 8 # cost mexico
c_c = 7.25 # cost china
i = 0.01 # interest rate
ini_bal = 0 # initial balance

## Single Sourcing: Mexico  

### Cumulative Average Forecast

In [ ]:
def single_source_mexico_cum_avg(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, sl):
    cash_balance = initial_cash_balance
    inventory = 0
    errors = []
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]
        
        # Calculate cumulative average demand up to the current period
        if period > 1:
            cum_avg_demand = np.ceil(np.mean(data.loc[data['period'] <= period, 'demand']))
        else:
            cum_avg_demand = np.ceil(current_demand)  # For the first period, use the current demand
        
        beginning_inventory = inventory
        
        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0

        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue
        
        # Determine order quantity based on cumulative average demand and current inventory
        order_quantity = 0
        order_cost = 0
        
        # Calculate error and update errors list
        forecast_error = current_demand - cum_avg_demand
        errors.append(forecast_error)
            
        # Calculate the specified percentile of errors
        if len(errors) > 0:
            error_percentile = np.percentile(errors, sl)
        else:
            error_percentile = 0

        # Adjust order quantity based on forecast and error percentile
        adjusted_forecast = cum_avg_demand + error_percentile
        if adjusted_forecast >= inventory:
            order_quantity = int(adjusted_forecast - inventory)
        else:
            order_quantity = 0

        order_cost = order_quantity * sourcing_cost
        cash_balance -= order_cost
        current_cash = cash_balance # record current balance
            
        ending_inventory = inventory

        # Update inventory with new order
        inventory += order_quantity
        
        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)
        
        # Debugging outputs (commented out)
        """
        print(f"Period: {period}")
        print(f"Beginning Inventory: {beginning_inventory}")
        print(f"Current Demand: {current_demand}")
        print(f"Sales: {sales}")
        print(f"Revenue: {revenue}")
        print(f"Order Quantity: {order_quantity}")
        print(f"Order Cost: {order_cost}")
        print(f"Ending Inventory: {ending_inventory}")
        print(f"Profit from Operation: {revenue - order_cost}")
        print(f"Profit from Interest: {cash_balance - current_cash}")
        print(f"Cash Balance: {cash_balance}")
        print("-" * 40)
        """
    
    return cash_balance

# Optimization function to find the best service level
def optimize_service_level(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, percentile_thresholds):
    best_sl = None
    best_value = -np.inf
    
    for sl in percentile_thresholds:
        ending_bank_account_value = single_source_mexico_cum_avg(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, sl)
        if ending_bank_account_value > best_value:
            best_value = ending_bank_account_value
            best_sl = sl
    
    return best_sl, best_value

# Define range of service levels to test (percentile thresholds)
percentile_thresholds = np.linspace(1, 99, 99)

# Optimize service level
best_sl_1, best_value_1 = optimize_service_level(data, s, c_m, i, ini_bal, percentile_thresholds)

# Print the best service level
print(f"Best Service Level: {best_sl_1:.0f}")

### Moving Average Forecast

In [ ]:
def single_source_mexico_mov_avg(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, moving_avg_window, sl):
    cash_balance = initial_cash_balance
    inventory = 0
    errors = []
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]

        # Calculate moving average demand over the specified window
        if period >= moving_avg_window:
            moving_avg_demand = np.ceil(np.mean(data.loc[(data['period'] > period - moving_avg_window) & (data['period'] <= period), 'demand']))
        else:
            moving_avg_demand = np.ceil(np.mean(data.loc[data['period'] <= period, 'demand']))

        beginning_inventory = inventory
        
        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0
        
        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue
        
        # Determine order quantity based on moving average demand and current inventory
        order_quantity = 0
        order_cost = 0
        if period >= moving_avg_window:
            # Calculate error and update errors list
            forecast_error = current_demand - moving_avg_demand
            errors.append(forecast_error)
            
            # Calculate the specified percentile of errors
            if len(errors) > 0:
                error_percentile = np.percentile(errors, sl)
            else:
                error_percentile = 0

            # Adjust order quantity based on forecast and error percentile
            adjusted_forecast = moving_avg_demand + error_percentile
            if adjusted_forecast >= inventory:
                order_quantity = int(adjusted_forecast - inventory)
            else:
                order_quantity = 0

            order_cost = order_quantity * sourcing_cost
            cash_balance -= order_cost
            current_cash = cash_balance # record current balance
            
            ending_inventory = inventory

            # Update inventory with new order
            inventory += order_quantity
        else:
            ending_inventory = inventory

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

        '''
        print(f"Period: {period}")
        print(f"Beginning Inventory: {beginning_inventory}")
        print(f"Current Demand: {current_demand}")
        print(f"Sales: {sales}")
        print(f"Revenue: {revenue}")
        print(f"Order Quantity: {order_quantity}")
        print(f"Order Cost: {order_cost}")
        print(f"Ending Inventory: {ending_inventory}")
        print(f"Profit from Operation: {revenue - order_cost}")
        print(f"Profit from Interest: {cash_balance - current_cash}")
        print(f"Cash Balance: {cash_balance}")
        print(f"Forecast Error: {forecast_error}")
        print(f"95th Percentile of Errors: {error_percentile}")
        print(f"Adjusted Forecast: {adjusted_forecast}")
        print("-" * 40)
        '''
    
    return cash_balance

# Optimization function to find the best service level
def optimize_service_level(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, moving_avg_window, percentile_thresholds):
    best_sl = None
    best_value = -np.inf
    
    for sl in percentile_thresholds:
        ending_bank_account_value = single_source_mexico_mov_avg(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, moving_avg_window, sl)
        if ending_bank_account_value > best_value:
            best_value = ending_bank_account_value
            best_sl = sl
    
    return best_sl, best_value

# Define range of service levels to test (percentile thresholds)
percentile_thresholds = np.linspace(1, 99, 99)

# Optimize service level
window = 10
best_sl_2, best_value_2 = optimize_service_level(data, s, c_m, i, ini_bal , window, percentile_thresholds)

# Print the best service level and the corresponding ending bank account value
print(f"Best Service Level: {best_sl_2:.0f}")

### Exponential Smoothing

In [ ]:
"""
from itertools import product

def single_source_mexico_exp_smt(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, alpha, sl):
    cash_balance = initial_cash_balance
    inventory = 0
    errors = []
    forecast = None
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]
        
        # Calculate exponential smoothing forecast
        if period == 1:
            forecast = current_demand  # Use the first period's demand as the initial forecast
        else:
            forecast = alpha * current_demand + (1 - alpha) * forecast

        # Calculate forecast error and update errors list
        if period > 1:
            forecast_error = current_demand - forecast
            errors.append(forecast_error)
            
            # Calculate the specified percentile of errors
            if len(errors) > 0:
                error_percentile = np.percentile(errors, sl)
            else:
                error_percentile = 0

            # Adjust order quantity based on forecast and error percentile
            adjusted_forecast = forecast + error_percentile
            if adjusted_forecast >= inventory:
                order_quantity = int(adjusted_forecast - inventory)
            else:
                order_quantity = 0

            order_cost = order_quantity * sourcing_cost
            cash_balance -= order_cost
            current_cash = cash_balance  # Record current balance
            
            # Record ending inventory before the new order arrives
            ending_inventory = inventory

            # Update inventory with new order
            inventory += order_quantity
        else:
            ending_inventory = inventory

        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0
        
        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

    return cash_balance

# Vectorized optimization function
def optimize_alpha_and_service_level_vectorized(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, alpha_values, service_levels):
    results = []
    
    # Create a grid of all combinations of alpha and service level
    grid = list(product(alpha_values, service_levels))
    
    for alpha, sl in grid:
        ending_bank_account_value = single_source_mexico_exp_smt(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, alpha, sl)
        results.append((alpha, sl, ending_bank_account_value))
    
    # Convert results to a DataFrame
    results_df = pd.DataFrame(results, columns=['Alpha', 'Service_Level', 'Ending_Bank_Account_Value'])
    
    # Find the best combination of alpha and service level
    best_result = results_df.loc[results_df['Ending_Bank_Account_Value'].idxmax()]
    
    return best_result['Alpha'], best_result['Service_Level'], best_result['Ending_Bank_Account_Value']

# Define range of alpha values and service levels to test
alpha_values = np.linspace(0.01, 0.99, 99)
service_levels = np.linspace(1, 99, 99)

# Optimize alpha and service level
best_alpha, best_sl_3, best_value_3 = optimize_alpha_and_service_level_vectorized(data, s, c_m, i, ini_bal, alpha_values, service_levels)

# Print the best alpha, best service level, and the corresponding ending bank account value
print(f"Best Alpha: {best_alpha:.2f}")
print(f"Best Service Level: {best_sl_3:.0f}")
"""

In [5]:
from joblib import Parallel, delayed
from itertools import product

def single_source_mexico_exp_smt(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, alpha, sl):
    cash_balance = initial_cash_balance
    inventory = 0
    errors = []
    forecast = None
    
    for period in data['period']:
        current_demand = data.loc[data['period'] == period, 'demand'].values[0]
        
        # Calculate exponential smoothing forecast
        if period == 1:
            forecast = current_demand
        else:
            forecast = alpha * current_demand + (1 - alpha) * forecast

        # Calculate forecast error and update errors list
        if period > 1:
            forecast_error = current_demand - forecast
            errors.append(forecast_error)
            
            # Calculate the specified percentile of errors
            if len(errors) > 0:
                error_percentile = np.percentile(errors, sl)
            else:
                error_percentile = 0

            # Adjust order quantity based on forecast and error percentile
            adjusted_forecast = forecast + error_percentile
            if adjusted_forecast >= inventory:
                order_quantity = int(adjusted_forecast - inventory)
            else:
                order_quantity = 0

            order_cost = order_quantity * sourcing_cost
            cash_balance -= order_cost
            current_cash = cash_balance  # Record current balance
            
            # Record ending inventory before the new order arrives
            ending_inventory = inventory

            # Update inventory with new order
            inventory += order_quantity
        else:
            ending_inventory = inventory

        # Satisfy demand from inventory
        if inventory >= current_demand:
            sales = current_demand
            inventory -= sales
        else:
            sales = inventory
            inventory = 0
        
        # Calculate revenue
        revenue = sales * sales_price
        cash_balance += revenue

        # Apply interest
        cash_balance = cash_balance * (1 + interest_rate)

    return cash_balance

# Parallelized optimization function
def optimize_alpha_and_service_level_parallel(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, alpha_values, service_levels):
    def evaluate_combination(alpha, sl):
        return (alpha, sl, single_source_mexico_exp_smt(data, sales_price, sourcing_cost, interest_rate, initial_cash_balance, alpha, sl))
    
    # Create a grid of all combinations of alpha and service level
    grid = list(product(alpha_values, service_levels))
    
    # Use parallel processing to evaluate combinations
    results = Parallel(n_jobs=-1)(delayed(evaluate_combination)(alpha, sl) for alpha, sl in grid)
    
    # Convert results to a DataFrame
    results_df = pd.DataFrame(results, columns=['Alpha', 'Service_Level', 'Ending_Bank_Account_Value'])
    
    # Find the best combination of alpha and service level
    best_result = results_df.loc[results_df['Ending_Bank_Account_Value'].idxmax()]
    
    return best_result['Alpha'], best_result['Service_Level'], best_result['Ending_Bank_Account_Value']

# Define range of alpha values and service levels to test
alpha_values = np.linspace(0.01, 0.99, 99)
service_levels = np.linspace(1, 99, 99)

# Optimize alpha and service level using parallel processing
best_alpha, best_sl_3, best_value_3 = optimize_alpha_and_service_level_parallel(data, s, c_m, i, ini_bal, alpha_values, service_levels)

# Print the best alpha, best service level, and the corresponding ending bank account value
print(f"Best Alpha: {best_alpha:.2f}")
print(f"Best Service Level: {best_sl_3:.2f}")
print(f"Ending Bank Account Value with Best Alpha and Service Level: ${best_value_3:,.2f}")

Best Alpha: 0.99
Best Service Level: 99.00
Ending Bank Account Value with Best Alpha and Service Level: $155,614,027,573,236,494,485,233,099,935,839,117,690,464,632,832.00


### Results

Results without optimization: \
Cumulative Average Forecast: $1.24e+47 \
10-Day Moving Average Forecast (95% Service Level): $1.28e+47 \
Exponential Smoothing Forecast (95% Service Level): $1.55e+47 

Results with optimization: \
Cumulative Average Forecast with 96% Service Level: $1.42e+47 \
10-Day Moving Average Forecast with 99% Service Level: $1.29e+47

In [7]:
# With Service Level Optimization
#print(f"Cumulative Average Forecast with {best_sl_1:.0f}% Service Level: ${best_value_1:,.2e}")
#print(f"10-Day Moving Average Forecast with {best_sl_2:.0f}% Service Level: ${best_value_2:,.2e}")
print(f"Exponential Smoothing Forecast with Alpha {best_alpha} and {best_sl_3}% Service Level: ${best_value_3:,.2e}")

Exponential Smoothing Forecast with Alpha 0.99 and 99.0% Service Level: $1.56e+47


## Single Sourcing: China

## Duo Sourcing Strategy